In [1]:
import gym
import gym_act

In [2]:
env = gym.make("Act-v0")

[2019-03-05 15:52:30,292] Making new env: Act-v0


SEED 5201950305025937922
[100   0   0  20   7  94  11   0  33  55  22   2  25  32  23   4   1  26
  18   4  25 173  24   1 199 158 -13  -2 167 116 -11  -4 181  38 -21  -4
 152 188 -15  -4 199  59 -20  -2]


In [3]:
action = 0
obs, reward, done, info = env.step(action)
img = env.render()

In [4]:
print(reward)

-86.76003979262714
